# DRL Portfolio Optimization


### Acknowledgements
This workbook is the culmination of three separate half credit independent studies by the author [Daniel Fudge](https://www.linkedin.com/in/daniel-fudge) with [Professor Yelena Larkin](https://www.linkedin.com/in/yelena-larkin-6b7b361b/) 
as part of a concurrent Master of Business Administration (MBA) and a [Diploma in Financial Engineering](https://schulich.yorku.ca/programs/fnen/) from the [Schulich School of Business](https://schulich.yorku.ca/).  I wanted to thank Schulich and especially Professor Larkin for giving me the freedom to explorer the intersection of Machine Learning and Finance.   

I'd like to also mention the excellent training I recieved from [A Cloud Guru](https://acloud.guru/learn/aws-certified-machine-learning-specialty) to prepare for this project.  

This notebook takes much of the design and low level code from the AWS sample portfolio management [Notebook](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/reinforcement_learning/rl_portfolio_management_coach_customEnv), which inturn is based on Jiang, Zhengyao, Dixing Xu, and Jinjun Liang. "A deep reinforcement learning framework for the financial portfolio management problem." arXiv preprint arXiv:1706.10059 (2017).    

As detailed below, this notebook relies on the [RL Coach](https://github.com/NervanaSystems/coach#batch-reinforcement-learning) from Intel AI Labs and OpenAI [Gym](https://gym.openai.com/).  Both of which are amazing projects that I highly recommend investigating.

### Introduction

Now that we have the signals `signals.pkl` and prices `stock-data-clean.pkl` processed we can build the Reinforcement Learning algorithm.  The signals will be used to define the state of the market environment, called `observations` in the Gym documentation and `state` in other locations.  The prices will be used to generate the `rewards`.   

For a refresher on DRL I recommend looking through [report2](docs/report2.pdf) in this repo.  The basic RL feed back loop is shown below.    

![RL](docs/rl.png "DDGP")

Reinforcement Learning feedback loop.    
Image source: https://i.stack.imgur.com/eoeSq.png

### Action Space
The `action` space is simply vector containing the weights of the stocks in the portfolio.  Inside the environment these weights are limited to (0, 1) and an addition weight is added for the amount of cash in the portfolio.  The cash weight is simply one minus the sum of the other weights and also limited to (0, 1).  The last step is to normalize these weight so the sum is equal to 1.0.

### State (or Observation)
The `state` is simply the signals we compiled in the previous data preparation [notebook](docs/data-preparation-no-memory.ipynb) including a time embbeding called the `window_length` within the code.  Instead of using the LSTM discussed previously, we are using a short Convolutional Neural Net (CNN) to capture some historical information from the data within the `agent`.

### Agent
For the agent we are using the [RL Coach](https://github.com/NervanaSystems/coach#batch-reinforcement-learning) from Intel AI Labs that is integrated into AWS [Sagemaker](https://docs.aws.amazon.com/sagemaker/latest/dg/reinforcement-learning.html#sagemaker-rl).  As you can see below there are a large number of RL algorithms to choose from.  In  [report2](docs/report2.pdf) we discussed serval of these and focused on the Deep Deterministic Policy Gradient (DDPG) Actor-Critic method however for this test we are trying the Proximal Policy Optimization (PPO) algorithm that is explained nicely by Jonathan Hui [here](https://medium.com/@jonathan_hui/rl-proximal-policy-optimization-ppo-explained-77f014ec3f12) and the original paper by Schulman et al. [here](https://arxiv.org/pdf/1707.06347.pdf).    

Remember that the goal of the agent is to learn a policy that maximizes the sum of discounted rewards, which in our case will result in the maximization of the portfolio value.

![Coach](docs/rl-coach.png)
RL Coach Algorithms
Imafe Source:  https://github.com/NervanaSystems/coach#batch-reinforcement-learning

### Environment
This leaves us with the environment to generate.  Here we build a custom finacial market environment (or simulator) based on the signals and prices we compiled previously on top of the OpenAI [Gym](https://gym.openai.com/), which is also integrated into AWS [Sagemaker](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-rl-environments.html#sagemaker-rl-environments-gym).  This takes care of the low level integration allows us to focus on the features of the environment specific to portfolio optimization.   

During each training epoch, the trainer randomly selects a date to start.  It then steps through each day in the epoch and following these high level operations.
1. Initialze the state from the signals with the randomly selected start date and sets the portfolio to $1 of cash.
1. Pass the state to the agent who generates an action, which is a new set of desired portfolio weights.
1. Pass the new weigths (action) to the environment who calculates: 
  * The new portfolio value based on changes in the prices, weights and transaction costs.   
  * The reward based on the previous weights and the change in prices.
  * The new state, which is simply pulled from the signals dataset.
1. Pass the new reward and state to the agent, who must then learn to make a better action.

## Setup
### Roles and permissions

To get started, we'll import the Python libraries we need, set up the environment with a few prerequisites for permissions and configurations.

In [19]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
from IPython.display import HTML
import time
from time import gmtime, strftime
sys.path.append("common")
from src.misc import get_execution_role, wait_for_s3_object
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

### Setup S3 buckets

Set up the linkage and authentication to the S3 bucket that you want to use for checkpoint and the metadata. 

In [12]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

S3 bucket path: s3://sagemaker-us-east-1-031118886020/


### Create an IAM role
Get the execution IAM role that allows this notebook to connect to the training and evaluation instances.

In [14]:
role = sagemaker.get_execution_role()
print("Using IAM role arn: {}".format(role))

Using IAM role arn: arn:aws:iam::031118886020:role/sagemaker


## Train the RL model using the Python SDK Script mode

1. Specify the source directory where the environment, presets and training code is uploaded, `src`.
2. Specify the `entry_point` as the training code,  
3. Specify the choice of RL `toolkit` and framework. This automatically resolves to the ECR path for the RL Container. 
4. Define the training parameters such as the instance count, job name, S3 path for output and job name. 
5. Specify the hyperparameters for the RL agent algorithm. The `RLCOACH_PRESET` can be used to specify the RL agent algorithm you want to use. 
6. [Optional] Choose the metrics that you are interested in capturing in your logs. These can also be visualized in CloudWatch and SageMaker Notebooks. The metrics are defined using regular expression matching.


In [21]:
job_name_prefix = 'drl-portfolio-optimization'
estimator = RLEstimator(source_dir='src',
                      entry_point="train-coach.py",
                      dependencies=["common/sagemaker_rl"],
                      toolkit=RLToolkit.COACH,
                      toolkit_version='0.11.0',
                      framework=RLFramework.MXNET,
                      role=role,
                      train_instance_count=1,
                      train_instance_type="ml.m4.4xlarge",
                      output_path=s3_output_path,
                      base_job_name=job_name_prefix,
                      hyperparameters = {
                          "RLCOACH_PRESET" : "preset-portfolio-management-clippedppo",
                          "rl.agent_params.algorithm.discount": 0.9,
                          "rl.evaluation_steps:EnvironmentEpisodes": 5
                      }
                    )
estimator.fit()

2020-03-26 01:42:04 Starting - Starting the training job...
2020-03-26 01:42:06 Starting - Launching requested ML instances......
2020-03-26 01:43:08 Starting - Preparing the instances for training...
2020-03-26 01:43:52 Downloading - Downloading input data...
2020-03-26 01:44:32 Training - Training image download completed. Training in progress.
2020-03-26 01:44:32 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-26 01:44:22,419 sagemaker-containers INFO     Imported framework sagemaker_mxnet_container.training
2020-03-26 01:44:22,422 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-03-26 01:44:22,437 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HPS': '{"RLCOACH_PRESET":"preset-portfolio-management-clippedppo","rl.agent_params.algorithm.discount":0.9,"rl.evaluation_steps:EnvironmentEpisodes":5}', 'SM_CURRE


2020-03-26 01:44:39 Failed - Training job failed


UnexpectedStatusException: Error for Training job drl-portfolio-optimization-2020-03-26-01-42-04-360: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python -m train-coach --RLCOACH_PRESET preset-portfolio-management-clippedppo --rl.agent_params.algorithm.discount 0.9 --rl.evaluation_steps:EnvironmentEpisodes 5"

## Store intermediate training output and model checkpoints 

The output from the training job above is either stored in a local directory (`local` mode) or on S3 (`SageMaker`) mode.


In [ ]:
%%time

job_name=estimator._current_job_name
print("Job name: {}".format(job_name))

s3_url = "s3://{}/{}".format(s3_bucket,job_name)
output_tar_key = "{}/output/output.tar.gz".format(job_name)

intermediate_folder_key = "{}/output/intermediate/".format(job_name)
output_url = "s3://{}/{}".format(s3_bucket, output_tar_key)
intermediate_url = "s3://{}/{}".format(s3_bucket, intermediate_folder_key)

print("S3 job path: {}".format(s3_url))
print("Output.tar.gz location: {}".format(output_url))
print("Intermediate folder path: {}".format(intermediate_url))
    
tmp_dir = "/tmp/{}".format(job_name)
os.system("mkdir {}".format(tmp_dir))
print("Create local folder {}".format(tmp_dir))

In [ ]:
%%time

wait_for_s3_object(s3_bucket, output_tar_key, tmp_dir)  

if not os.path.isfile("{}/output.tar.gz".format(tmp_dir)):
    raise FileNotFoundError("File output.tar.gz not found")
os.system("tar -xvzf {}/output.tar.gz -C {}".format(tmp_dir, tmp_dir))
if not local_mode:
    os.system("aws s3 cp --recursive {} {}".format(intermediate_url, tmp_dir))
if not os.path.isfile("{}/output.tar.gz".format(tmp_dir)):
    raise FileNotFoundError("File output.tar.gz not found")
os.system("tar -xvzf {}/output.tar.gz -C {}".format(tmp_dir, tmp_dir))
print("Copied output files to {}".format(tmp_dir))

if local_mode:
    checkpoint_dir = "{}/data/checkpoint".format(tmp_dir)
    info_dir = "{}/data/".format(tmp_dir)
else:
    checkpoint_dir = "{}/checkpoint".format(tmp_dir)
    info_dir = "{}/".format(tmp_dir)

print("Checkpoint directory {}".format(checkpoint_dir))
print("info directory {}".format(info_dir))

## Visualization

### Plot rate of learning

We can view the rewards during training using the code below. This visualization helps us understand how the performance of the model represented as the reward has improved over time. For the consideration of training time, we restict the episodes number. If you see the final reward (average logarithmic cumulated return) is still below zero, try a larger training steps. The number of steps can be configured in the preset file.

In [ ]:
%matplotlib inline
import pandas as pd

csv_file_name = "worker_0.simple_rl_graph.main_level.main_level.agent_0.csv"
key = os.path.join(intermediate_folder_key, csv_file_name)
wait_for_s3_object(s3_bucket, key, tmp_dir)

csv_file = "{}/{}".format(tmp_dir, csv_file_name)
df = pd.read_csv(csv_file)
df = df.dropna(subset=['Evaluation Reward'])
# print(list(df))
x_axis = 'Episode #'
y_axis = 'Evaluation Reward'

plt = df.plot(x=x_axis,y=y_axis, figsize=(12,5), legend=True, style='b-')
plt.set_ylabel(y_axis);
plt.set_xlabel(x_axis);

### Visualize the portfolio value

We use result of the last evaluation phase as an example to visualize the portfolio value. The following figure demonstrates reward vs date. Sharpe ratio and maximum drawdown are also calculated to help readers understand the return of an investment compared to its risk. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# same as in https://github.com/vermouth1992/drl-portfolio-management/blob/master/src/environment/portfolio.py
def sharpe(returns, freq=30, rfr=0):
    """ Given a set of returns, calculates naive (rfr=0) sharpe. """
    eps = np.finfo(np.float32).eps
    return (np.sqrt(freq) * np.mean(returns - rfr + eps)) / np.std(returns - rfr + eps)


def max_drawdown(returns):
    """ Max drawdown. See https://www.investopedia.com/terms/m/maximum-drawdown-mdd.asp """
    eps = np.finfo(np.float32).eps
    peak = returns.max()
    trough = returns[returns.argmax():].min()
    return (trough - peak) / (peak + eps)


In [ ]:
info = info_dir + 'portfolio-management.csv'
df_info = pd.read_csv(info)
df_info['date'] = pd.to_datetime(df_info['date'], format='%Y-%m-%d')
df_info.set_index('date', inplace=True)
mdd = max_drawdown(df_info.rate_of_return + 1)
sharpe_ratio = sharpe(df_info.rate_of_return)
title = 'max_drawdown={: 2.2%} sharpe_ratio={: 2.4f}'.format(mdd, sharpe_ratio)
df_info[["portfolio_value", "market_value"]].plot(title=title, fig=plt.gcf(), rot=30)

## Load the checkpointed models for evaluation

Checkpointed data from the previously trained models will be passed on for evaluation / inference in the `checkpoint` channel. 

In [ ]:
%%time

checkpoint_path = "s3://{}/{}/checkpoint/".format(s3_bucket, job_name)
if not os.listdir(checkpoint_dir):
    raise FileNotFoundError("Checkpoint files not found under the path")
os.system("aws s3 cp --recursive {} {}".format(checkpoint_dir, checkpoint_path))
print("S3 checkpoint file path: {}".format(checkpoint_path))

### Run the evaluation step

Use the checkpointed model to run the evaluation step. 


In [ ]:
%%time

estimator_eval = RLEstimator(role=role,
                      source_dir='src/',
                      dependencies=["common/sagemaker_rl"],
                      toolkit=RLToolkit.COACH,
                      toolkit_version='0.11.0',
                      framework=RLFramework.MXNET,
                      entry_point="evaluate-coach.py",
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      hyperparameters = {
                          "evaluate_steps": 731*2 # evaluate on 2 episodes
                      }
                    )
estimator_eval.fit({'checkpoint': checkpoint_path})

## Risk Disclaimer (for live-trading)

This notebook is for educational purposes only. Past trading performance does not guarantee future performance. The loss in trading can be substantial, and therefore 
**investors should use all trading strategies at their own risk**.